<table align="left">
  <tr>
    <td align="left"><h1>3 : Monitoring and Evaluating your Agent</h1></td>
  </tr>
</table>


## Setup Tracing

In [1]:
PROJECT_NAME = "Customer-Success"

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

tracer_provider = register(
    project_name=PROJECT_NAME,
    #endpoint= get_phoenix_endpoint() + "v1/traces"
    endpoint = os.getenv('DLAI_LOCAL_URL').format(port='6006') + "v1/traces"
)
SmolagentsInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Customer-Success
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://s172-29-95-145p6006.lab-aws-production.deeplearning.ai/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv() # load variables from local .env file

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

In [4]:
from smolagents import HfApiModel

model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

model([{"role": "user", "content": "Hello!"}])

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Hello! How can I assist you today?', tool_calls=[], raw=ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Hello! How can I assist you today?', tool_call_id=None, tool_calls=[]), logprobs=None, seed=13618980564928662000)], created=1745329835, id='nr7pchw-4yUbBN-9345974f3e93058b', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=ChatCompletionOutputUsage(completion_tokens=10, prompt_tokens=31, total_tokens=41), object='chat.completion', prompt=[]))

In [5]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-95-145p6006.lab-aws-production.deeplearning.ai/


## Trace an agent run

In [6]:
from smolagents import HfApiModel, CodeAgent

agent = CodeAgent(model=model, tools=[])

>Note, the following line will sometimes get a timeout on the interface to the tracing package due to the networked interface. If this happens, try it again.


In [7]:
agent.run("What is the 100th Fibonacci number?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the 100th Fibonacci number?                                                                             │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n):                                                                                           
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fib_100 = fibonacci(100)                                                                                         
  final_answer(fib_100)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 354224848179261915075

[Step 1: Duration 0.27 seconds| Input tokens: 2,020 | Output tokens: 130]

354224848179261915075

In [8]:
# This is where you can access the display:
print(os.environ.get('DLAI_LOCAL_URL').format(port='6006'))

https://s172-29-95-145p6006.lab-aws-production.deeplearning.ai/


## Setup ice cream production system

In [9]:
from smolagents import tool
from typing import Dict

menu_prices = {"crepe nutella": 1.50, "vanilla ice cream": 2, "maple pancake": 1.}

ORDER_BOOK = {}

@tool
def place_order(quantities: Dict[str, int], session_id: int) -> None:
    """Places a pre-order of snacks.

    Args:
        quantities: a dictionary with names as keys and quantities as values
        session_id: the id for the client session
    """
    global ORDER_BOOK
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    ORDER_BOOK[session_id] = quantities

@tool
def get_prices(quantities: Dict[str, int]) -> str:
    """Gets price for certain quantities of ice cream.

    Args:
        quantities: a dictionary with names as keys and quantities as values
    """
    assert isinstance(quantities, dict), "Incorrect type for the input dictionary!"
    assert [key in menu_prices for key in quantities.keys()], f"All food names should be within {menu_prices.keys()}"
    total_price = sum([menu_prices[key] * value for key, value in quantities.items()])
    return (
        f"Given the current menu prices:\n{menu_prices}\nThe total price for your order would be: ${total_price}"
    )

In [10]:
order_agent = CodeAgent(
    tools=[place_order, get_prices],
    model=HfApiModel("Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")
)

In [11]:
order_agent.run(
    "Could I come and collect one crepe nutella?",
    additional_args={"session_id": 192}
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 192}.                                                                                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  place_order(quantities={'crepe nutella': 1}, session_id=session_id)                                              
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.11 seconds| Input tokens: 2,193 | Output tokens: 68]

'Order placed for one crepe nutella.'

### Try multiple orders

In [12]:
client_requests = [
    ("Could I come and collect one crepe nutella?", "place_order"),
    ("What would be the price for 1 crêpe nutella + 2 pancakes?", "get_prices"),
    ("How did you start your ice-cream business?", None),
    ("What's the weather at the Louvre right now?", None),
    ("I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want it. If it's below, I'll order it, please.", "place_order")
]

In [13]:
for request in client_requests:
    order_agent.run(
        request[0],
        additional_args={"session_id": 0, "menu_prices": menu_prices}
    )

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Could I come and collect one crepe nutella?                                                                     │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  quantities = {'crepe nutella': 1}                                                                                
  place_order(quantities=quantities, session_id=session_id)                                                        
  final_answer("Order placed for one crepe nutella.")                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Order placed for one crepe nutella.

[Step 1: Duration 0.10 seconds| Input tokens: 2,227 | Output tokens: 93]

...

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm not sure if I should order. I want a vanilla ice cream. but if it's more expensive than $1, I don't want    │
│ it. If it's below, I'll order it, please.                                                                       │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'session_id': 0, 'menu_prices': {'crepe nutella': 1.5, 'vanilla ice cream': 2, 'maple pancake': 1.0}}.         │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  vanilla_ice_cream_price = menu_prices['vanilla ice cream']                                                       
  print(f"The price of vanilla ice cream is ${vanilla_ice_cream_price}.")                                          
                                                                                                                   
  final_answer("I will not order the vanilla ice cream as it is more than $1.")                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The price of vanilla ice cream is $2.

Out - Final answer: I will not order the vanilla ice cream as it is more than $1.

[Step 1: Duration 0.08 seconds| Input tokens: 2,260 | Output tokens: 162]

In [14]:
import phoenix as px

spans = px.Client().get_spans_dataframe(project_name=PROJECT_NAME)
spans.head(20)

/usr/local/lib/python3.11/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (8.29.0) and client (8.19.1) versions are mismatched and may have compatibility issues.
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.output.mime_type,attributes.llm.token_count.total,attributes.openinference.span.kind,attributes.llm.token_count.completion,attributes.llm.token_count.prompt,attributes.llm.input_messages,attributes.tool.name,attributes.tool.parameters,attributes.tool.description,attributes.smolagents
context.span_id,,,,,,,,,,,,,,,,,,,,,
4eb597f99afb35e4,HfApiModel.__call__,LLM,None,2025-04-29 05:19:30.694881+00:00,2025-04-29 05:19:30.867969+00:00,OK,,[],4eb597f99afb35e4,352fc8e06fc72aad300c59879a600b28,...,application/json,41.0,LLM,10.0,31.0,None,None,None,None,None
46d25ae357db6dae,HfApiModel.__call__,LLM,fa12c50334c05682,2025-04-29 05:19:30.963861+00:00,2025-04-29 05:19:30.982949+00:00,OK,,[],46d25ae357db6dae,fe38665c5f914c1cc366b36994139fcb,...,application/json,2150.0,LLM,130.0,2020.0,[{'message.content': 'You are an expert assist...,None,None,None,None
d7e5027b046cb217,FinalAnswerTool,TOOL,fa12c50334c05682,2025-04-29 05:19:31.186374+00:00,2025-04-29 05:19:31.186436+00:00,OK,,[],d7e5027b046cb217,fe38665c5f914c1cc366b36994139fcb,...,None,NaN,TOOL,NaN,NaN,None,final_answer,"{'answer': {'type': 'any', 'description': 'The...",Provides a final answer to the given problem.,None
fa12c50334c05682,Step 1,CHAIN,46a5bdca2cb0add0,2025-04-29 05:19:30.963671+00:00,2025-04-29 05:19:31.209107+00:00,OK,,[],fa12c50334c05682,fe38665c5f914c1cc366b36994139fcb,...,None,NaN,CHAIN,NaN,NaN,None,None,None,None,None
46a5bdca2cb0add0,CodeAgent.run,AGENT,None,2025-04-29 05:19:30.956528+00:00,2025-04-29 05:19:31.229395+00:00,OK,,[],46a5bdca2cb0add0,fe38665c5f914c1cc366b36994139fcb,...,None,2150.0,AGENT,130.0,2020.0,None,None,None,None,"{'max_steps': 20, 'tools_names': ['final_answe..."
e67dec26c1de05db,HfApiModel.__call__,LLM,93a164caf4b53c5a,2025-04-29 05:19:31.281373+00:00,2025-04-29 05:19:31.306439+00:00,OK,,[],e67dec26c1de05db,9252ae7ede18188e9e441ed70d1c336d,...,application/json,2261.0,LLM,68.0,2193.0,[{'message.content': 'You are an expert assist...,None,None,None,None
53bd88233e8ccb0f,SimpleTool,TOOL,93a164caf4b53c5a,2025-04-29 05:19:31.330963+00:00,2025-04-29 05:19:31.331010+00:00,OK,,[],53bd88233e8ccb0f,9252ae7ede18188e9e441ed70d1c336d,...,None,NaN,TOOL,NaN,NaN,None,place_order,"{'quantities': {'type': 'object', 'additionalP...",Places a pre-order of snacks.,None
6f1a2dc29bd1a081,FinalAnswerTool,TOOL,93a164caf4b53c5a,2025-04-29 05:19:31.350968+00:00,2025-04-29 05:19:31.351011+00:00,OK,,[],6f1a2dc29bd1a081,9252ae7ede18188e9e441ed70d1c336d,...,None,NaN,TOOL,NaN,NaN,None,final_answer,"{'answer': {'type': 'any', 'description': 'The...",Provides a final answer to the given problem.,None
93a164caf4b53c5a,Step 1,CHAIN,939d8279fbe73fb1,2025-04-29 05:19:31.281163+00:00,2025-04-29 05:19:31.372720+00:00,OK,,[],93a164caf4b53c5a,9252ae7ede18188e9e441ed70d1c336d,...,None,NaN,CHAIN,NaN,NaN,None,None,None,None,None


### Add processing to extract desired information

In [15]:
import pandas as pd
import json

agents = spans[spans['span_kind'] == 'AGENT'].copy()
agents['task'] = agents['attributes.input.value'].apply(
    lambda x: json.loads(x).get('task') if isinstance(x, str) else None
)

tools = spans.loc[
    spans['span_kind'] == 'TOOL',
    ["attributes.tool.name", "attributes.input.value", "context.trace_id"]
].copy()

tools_per_task = agents[
    ["name", "start_time", "task", "context.trace_id"]
].merge(
    tools,
    on="context.trace_id",
    how="left",
)
tools_per_task.head()

,name,start_time,task,context.trace_id,attributes.tool.name,attributes.input.value
0,CodeAgent.run,2025-04-29 05:19:30.956528+00:00,What is the 100th Fibonacci number?,fe38665c5f914c1cc366b36994139fcb,final_answer,"{""args"": [354224848179261915075], ""sanitize_in..."
1,CodeAgent.run,2025-04-29 05:19:31.275970+00:00,Could I come and collect one crepe nutella?,9252ae7ede18188e9e441ed70d1c336d,place_order,"{""args"": [], ""sanitize_inputs_outputs"": false,..."
2,CodeAgent.run,2025-04-29 05:19:31.275970+00:00,Could I come and collect one crepe nutella?,9252ae7ede18188e9e441ed70d1c336d,final_answer,"{""args"": [""Order placed for one crepe nutella...."
3,CodeAgent.run,2025-04-29 05:19:31.421743+00:00,Could I come and collect one crepe nutella?,71f88bd49f77abba06f1dd3148de6d32,place_order,"{""args"": [], ""sanitize_inputs_outputs"": false,..."
4,CodeAgent.run,2025-04-29 05:19:31.421743+00:00,Could I come and collect one crepe nutella?,71f88bd49f77abba06f1dd3148de6d32,final_answer,"{""args"": [""Order placed for one crepe nutella...."


### Now, compare tool calls with exected tool calls

In [16]:
def score_request(expected_tool: str, tool_calls: list):
    if expected_tool is None:
        return tool_calls == set(["final_answer"])
    else:
        return expected_tool in tool_calls

results = []
for request, expected_tool in client_requests:
    tool_calls = set(tools_per_task.loc[tools_per_task["task"] == request, "attributes.tool.name"].tolist())
    results.append(
        {
            "request": request,
            "tool_calls_performed": tool_calls,
            "is_correct": score_request(expected_tool, tool_calls)
        }
    )
pd.DataFrame(results)

,request,tool_calls_performed,is_correct
0,Could I come and collect one crepe nutella?,"{final_answer, place_order}",True
1,What would be the price for 1 crêpe nutella + ...,"{get_prices, final_answer}",True
2,How did you start your ice-cream business?,"{get_prices, final_answer}",False
3,What's the weather at the Louvre right now?,{final_answer},True
4,I'm not sure if I should order. I want a vanil...,{final_answer},False
